In [1]:
import spacy
import re
import sys
import random
sys.path.append("..")

from datasets.newsgroups import NewsgroupsDataset
from train import get_args

In [2]:
"""
Load dataset with custom arguments
Note: May take a couple mins
"""

class args:
    def __init__(self):
        default_args = get_args()
        
        self.dataset_dir = '../data/'
        self.toy = False
        self.file_batch_size = default_args.file_batch_size
        self.window_size = default_args.window_size
        self.nlp = None
        self.use_pretrained = False
        
dataset = NewsgroupsDataset(args())

  7%|▋         | 3/46 [00:00<00:01, 26.23it/s]

Loading dataset from: saved_datasets/20_news_groups_dataset/


100%|██████████| 46/46 [00:01<00:00, 26.59it/s]

Loaded Dataset!
There were 12746 tokens generated


In [4]:
"""
View a random file's tokenization
"""

rand_doc = random.choice(dataset.files)
tokenized_doc = dataset.tokenizer.tokenize_doc(rand_doc)

print(f'Tokenized Document:\n\n{tokenized_doc}')
print(f'\nRandom Document:\n\n{rand_doc}')

Tokenized Document:

['co', 'authored', 'report', 'gun', 'owners', 'action', 'league', 'state', 'rifle', 'association', 'started', 'day', 'rally', 'secluded', 'courtyard', 'statehouse', 'looking', 'sparse', 'about', 'people', 'speaker', 'began', 'about', 'people', 'followed', 'loudspeakers', 'lost', 'filled', 'area', 'proud', 'mikeyacino', 'goal', 'spoke', 'best', 'throwaway', 'lines', 'remind', 'holders', 'carry', 'permits', 'checked', 'certified', 'clear', 'crimes', 'state', 'people', 'statehouse', 'certified', 'clean', 'election', 'fraud', 'hold', 'jobs', 'nancysnow', 'amoshamburger', 'busy', 'handing', 'id', 'buttons', 'sheets', 'describing', 'bills', 'presented', 'hearings', 'telling', 'people', 'find', 'representatives', 'cases', 'mike', 'warned', 'committee', 'going', 'suspend', 'rules', 'discuss', 'bill', 'official', 'list', 'delegation', 'students', 'simon', 'rockofbardcollege', 'alma', 'mater', 'waynelo', 'shot', 'place', 'sks', 'latelastyear', 'bussed', 'testify', 'bill', 'b

In [44]:
def my_tokenize(doc_str, nlp):
    """
    Tokenize a document string
    Modified version of Moody's Tokenization in:
    https://github.com/cemoody/lda2vec/blob/master/lda2vec/preprocess.py

    :params doc_str: String
    :returns: list of Strings, i.e. tokens
    """

    # Send doc_str through pipeline
    spacy_doc = nlp(doc_str)
    # Filter Raw
    clean_doc = filter(is_valid_token, spacy_doc)
    # Convert to text make lowercase
    clean_doc = [t.text.lower() for t in spacy_doc]
    # Only allow characters in the alphabet and '_'
    clean_doc = [re.sub('[^a-zA-Z]', '', t) for t in clean_doc]
    # Remove any resulting empty indices
    clean_doc = [t for t in clean_doc if len(t) > 2]
    return clean_doc

def is_valid_token(token):
    """
    Determines if a token is valid or not

    :params token: String
    :returns: Boolean
    """
    if token.like_url:
        return False
    if token.like_email:
        return False
    if token.is_stop:
        return False

    return True
    

In [71]:
from spacy.util import filter_spans

def moodys_merge_noun_chunks(doc):
        """
        Merge noun chunks into a single token.
        
        Modified from sources of:
        - https://github.com/cemoody/lda2vec/blob/master/lda2vec/preprocess.py
        - https://spacy.io/api/pipeline-functions#merge_noun_chunks
        
        :params doc: Doc object.
        :returns: Doc object with merged noun chunks.
        """
        bad_deps = ('amod', 'compound')
        
        if not doc.is_parsed:
            return doc
        with doc.retokenize() as retokenizer:
            noun_chunks = filter_spans(list(doc.noun_chunks))
            for np in noun_chunks:
                
                # Only keep adjectives and nouns, e.g. "good ideas"
                while len(np) > 1 and np[0].dep_ not in bad_deps:
                    np = np[1:]
                
                if len(np) > 1:
                    # Merge NPs
                    attrs = {"tag": np.root.tag, "dep": np.root.dep}
                    retokenizer.merge(np, attrs=attrs)
                
        return doc
    

nlp = spacy.load("en_core_web_sm")
nlp.add_pipe(moodys_merge_noun_chunks, name="moody")
print(nlp.pipe_names)  # ['tagger', 'parser', 'ner', 'print_info']

['tagger', 'parser', 'ner', 'moody']


In [72]:
from tqdm import tqdm_notebook as tqdm
print(nlp.pipe_names)
doc = my_tokenize('Devin de Hueck rides within a mercedes benz iin the framework of django flask', nlp)
print(doc)

['tagger', 'parser', 'ner', 'moody']
['devindehueck', 'rides', 'within', 'mercedesbenz', 'iin', 'the', 'framework', 'djangoflask']
